Importando bibliotecas necessarias para manipualçao de dados


In [117]:
import pandas as pd
import numpy as np
import requests
import urllib.request
import os
from glob import glob

#Visualização de Dados
import seaborn as sb 
import matplotlib.pyplot as plt 

    
    """
7Fazendo aquisição de dados na pagina web 
Metodo e ferramentas de Extração utilizados 
- Lib resquests

    """"

In [11]:

arquivos_meses2018 = {
    "vra_122018": "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra_122018.csv",
    "vra_112018": "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra_112018.csv",
    "vra_102018": "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra_102018.csv",
    "vra_092018": "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra_092018.csv",
    "vra_082018": "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra_082018.csv",
    "vra_072018": "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra_072018.csv",
    "vra_062018": "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra_062018.csv",
    "vra_05.csv": "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra_05.csv",
    "vra_04.csv": "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra_04.csv",
    "vra_032018": "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra_032018.csv",
    "vra-02_2018": "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra-02_2018.csv",
    "vra-01_2018": "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2018/vra-01_2018.csv"
}




In [12]:
#fazendo requisição dos aquivos
#utilizando a biblioteca requests para acessar o pacote de dados que precisamos para manipulação 


for i, url_arquivo in enumerate(arquivos_meses2018.values(), start=1):
    caminho_destino = 'C:\\Users\\tonho\\OneDrive\\Documentos\\portifoliodedados\\webscaping\\PortifolioDeDados\\caseGOL\\{}.csv'.format(list(arquivos_meses2018.keys())[i-1])
    # response = requests.get(url_arquivo)
    with open(caminho_destino, 'wb') as file:
        file.write(requests.get(url_arquivo).content)
        print(f"Arquivo {i} baixado em: {caminho_destino}")


Arquivo 1 baixado em: C:\Users\tonho\OneDrive\Documentos\portifoliodedados\webscaping\PortifolioDeDados\caseGOL\vra_122018.csv
Arquivo 2 baixado em: C:\Users\tonho\OneDrive\Documentos\portifoliodedados\webscaping\PortifolioDeDados\caseGOL\vra_112018.csv
Arquivo 3 baixado em: C:\Users\tonho\OneDrive\Documentos\portifoliodedados\webscaping\PortifolioDeDados\caseGOL\vra_102018.csv
Arquivo 4 baixado em: C:\Users\tonho\OneDrive\Documentos\portifoliodedados\webscaping\PortifolioDeDados\caseGOL\vra_092018.csv
Arquivo 5 baixado em: C:\Users\tonho\OneDrive\Documentos\portifoliodedados\webscaping\PortifolioDeDados\caseGOL\vra_082018.csv
Arquivo 6 baixado em: C:\Users\tonho\OneDrive\Documentos\portifoliodedados\webscaping\PortifolioDeDados\caseGOL\vra_072018.csv
Arquivo 7 baixado em: C:\Users\tonho\OneDrive\Documentos\portifoliodedados\webscaping\PortifolioDeDados\caseGOL\vra_062018.csv
Arquivo 8 baixado em: C:\Users\tonho\OneDrive\Documentos\portifoliodedados\webscaping\PortifolioDeDados\caseGOL

Acessando arquivos 

 


In [3]:

import glob
Lista_meses2018 = glob.glob(r'C:\Users\tonho\OneDrive\Documentos\portifoliodedados\webscaping\PortifolioDeDados\caseGOL\*.csv')
# 'arquivos' agora é um array com o nome de todos os .csv que começam com 'arquivo'
lista_df_meses2018 = []
for arquivo_mes in Lista_meses2018:
   lista_df_meses2018.append(pd.read_csv(arquivo_mes,sep=';' , encoding='latin-1'))



C:\Users\tonho\AppData\Local\Temp\ipykernel_11316\2255298475.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  lista_df_meses2018.append(pd.read_csv(arquivo_mes,sep=';' , encoding='latin-1'))
C:\Users\tonho\AppData\Local\Temp\ipykernel_11316\2255298475.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  lista_df_meses2018.append(pd.read_csv(arquivo_mes,sep=';' , encoding='latin-1'))
C:\Users\tonho\AppData\Local\Temp\ipykernel_11316\2255298475.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  lista_df_meses2018.append(pd.read_csv(arquivo_mes,sep=';' , encoding='latin-1'))


In [4]:
for lista_datasets in lista_df_meses2018 :
    # Converta todas as colunas para 'category'
    lista_datasets2018 = lista_datasets.astype('object')

In [5]:
lista_datasets2018.head()

,ICAO Empresa Aérea,Número Voo,Código DI,Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Código Justificativa
0,AAL,213,0.0,I,KMIA,SBBR,01/12/2018 02:15,01/12/2018 03:11,01/12/2018 09:50,01/12/2018 10:35,REALIZADO,MX
1,AAL,213,0.0,I,KMIA,SBBR,02/12/2018 02:15,02/12/2018 02:15,02/12/2018 09:50,02/12/2018 09:50,REALIZADO,NaN
2,AAL,213,0.0,I,KMIA,SBBR,03/12/2018 02:15,03/12/2018 02:15,03/12/2018 09:50,03/12/2018 09:50,REALIZADO,NaN
3,AAL,213,0.0,I,KMIA,SBBR,04/12/2018 02:15,04/12/2018 02:15,04/12/2018 09:50,04/12/2018 09:50,REALIZADO,NaN
4,AAL,213,0.0,I,KMIA,SBBR,05/12/2018 02:15,05/12/2018 02:15,05/12/2018 09:50,05/12/2018 09:50,REALIZADO,NaN


In [71]:
df_aerodromos_publicos_nacionais = pd.read_csv('cadastro-de-aerodromos-civis-publicos.csv', usecols=['Aeródromo'], sep=';' , encoding='utf-8')

In [76]:
df_aerodromos_publicos_nacionais.drop(0, axis=0)

,Aeródromo
1,SBAA
2,SBAE
3,SBAQ
4,SBAR
5,SBAT
...,...
496,SJ4Y
497,SI68
498,SN6L
499,SBFS


Coleta de dados dos Aerodromos nacionais

In [77]:
lista_datasets2018 =lista_datasets2018[lista_datasets2018['ICAO Empresa Aérea'].isin(['GLO','AZU','TAM'])]

In [95]:
df_aerodromos_publicos_nacionais.drop(0, axis= 0, inplace=True)

In [96]:
df_aerodromos_publicos_nacionais.head()

,Aeródromo
1,SBAA
2,SBAE
3,SBAQ
4,SBAR
5,SBAT


#Estabelece uma condição para verificar se o aerodro de destino e chega são nacionais, o que configurara
#um voo nacional

In [88]:
condicao_origem = lista_datasets2018['ICAO Aeródromo Origem'].isin(df_aerodromos_publicos_nacionais['Aeródromo'])
condicao_destino = lista_datasets2018['ICAO Aeródromo Destino'].isin(df_aerodromos_publicos_nacionais['Aeródromo'])

# Aplicar as condições
lista_datasets2018 = lista_datasets2018[condicao_origem & condicao_destino]

# Resultado



In [89]:
lista_datasets2018.head()

,ICAO Empresa Aérea,Número Voo,Código DI,Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo
2518,AZU,2018,0.0,N,SBEG,SBBV,15/12/2018 22:00,15/12/2018 22:00,15/12/2018 23:20,15/12/2018 23:20,REALIZADO
2519,AZU,2018,0.0,N,SBKP,SBEG,15/12/2018 17:35,15/12/2018 17:31,15/12/2018 21:20,15/12/2018 21:17,REALIZADO
2520,AZU,2019,7.0,N,SBBR,SBKP,16/12/2018 05:05,16/12/2018 05:01,16/12/2018 06:40,16/12/2018 06:31,REALIZADO
2521,AZU,2019,9.0,N,SBBV,SBBR,16/12/2018 01:05,16/12/2018 01:05,16/12/2018 04:35,16/12/2018 04:35,REALIZADO
2522,AZU,2400,0.0,N,SBGR,SBRJ,01/12/2018 06:05,NaN,01/12/2018 07:10,NaN,CANCELADO


In [90]:
lista_datasets2018.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62943 entries, 2518 to 85487
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ICAO Empresa Aérea      62943 non-null  object
 1   Número Voo              62943 non-null  object
 2   Código DI               62943 non-null  object
 3   Código Tipo Linha       62943 non-null  object
 4   ICAO Aeródromo Origem   62943 non-null  object
 5   ICAO Aeródromo Destino  62943 non-null  object
 6   Partida Prevista        62943 non-null  object
 7   Partida Real            61768 non-null  object
 8   Chegada Prevista        62943 non-null  object
 9   Chegada Real            61768 non-null  object
 10  Situação Voo            62943 non-null  object
dtypes: object(11)
memory usage: 5.8+ MB


In [ ]:
lista_datasets2018.drop('Código Justificativa', axis=1, inplace=True)

In [91]:
lista_datasets2018.isnull().sum()

ICAO Empresa Aérea           0
Número Voo                   0
Código DI                    0
Código Tipo Linha            0
ICAO Aeródromo Origem        0
ICAO Aeródromo Destino       0
Partida Prevista             0
Partida Real              1175
Chegada Prevista             0
Chegada Real              1175
Situação Voo                 0
dtype: int64

In [92]:
lista_datasets2018.duplicated().sum()


0

Preparando as colunas horario Partida Prevista, Partida Real, Chegada Prevista, CHegada Real
As colunas estão disposta no formato data e hora, contudo no tipo object. A conversão para o tipo to_datetime vai facilitar na manipulação desses dados para geração de informações como as de pontualidade.

In [100]:
lista_datasets2018['Partida Prevista'] = pd.to_datetime(lista_datasets2018['Partida Prevista'], errors='coerce')
lista_datasets2018['Partida Real'] = pd.to_datetime(lista_datasets2018['Partida Real'], errors='coerce')
lista_datasets2018['Chegada Prevista'] = pd.to_datetime(lista_datasets2018['Chegada Prevista'], errors='coerce')
lista_datasets2018['Chegada Real'] = pd.to_datetime(lista_datasets2018['Chegada Real'], errors='coerce')


C:\Users\tonho\AppData\Local\Temp\ipykernel_11316\2460799032.py:4: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  lista_datasets2018['Chegada Real'] = pd.to_datetime(lista_datasets2018['Chegada Real'], errors='coerce')


In [103]:
lista_datasets2018['Partida Prevista'].dt.time

2518     22:00:00
2519     17:35:00
2520     05:05:00
2521     01:05:00
2522     06:05:00
           ...   
85431    18:33:00
85434    12:11:00
85479    16:17:00
85486    20:15:00
85487    20:15:00
Name: Partida Prevista, Length: 62943, dtype: object

Criando uma coluna "Status horario voo". Essa coluna vai guardar a indicação de voo "atrasado==1" ou "pontual == 0"

A construção dessa coluna será baseada em:
Um voo pode ser dado como pontual ou atrasado, levando-se em conta apenas os dados de 
“Partida Prevista somando 15 minuto” (Paertida_prevista + 15) e “Partida Real”, sendo:
Se Partida Real > Partida Prevista + 15 minutos → Voo atrasado (1)
Se Partida Real <= Partida Prevista + 15 minutos → Voo pontual(0)

No codigo baixo, o iterrows(): intera sobre todas as colunas do cojunto de dados fazendo a verificação da condição estabelecidade no bloco condional if

lista_datasets2018.at[i, "Status horario voo"] : correponde a uma nova coluna em que cada linha "i" recebe um rotulo para o voo de acordo com a metrica disponibilizada acima.

lista_datasets2018.at[i, "Status horario voo"] = np.nan: recebe NaN pois a partida real do voo está sem valor (NaN).

As condições são baseadas na existencia de um valor da linha na linha 'Partida Real' o que pode servir como premissa de garantia de que o voo realmente aconteceu.

In [119]:
import pandas as pd

# Supondo que 'Status horario voo' seja uma coluna existente em lista_datasets2018

for i, row in lista_datasets2018.iterrows():
    if row['Partida Real'] > row['Partida Prevista'] + pd.Timedelta(minutes=15):
        #Voo atrasado (1)
        lista_datasets2018.at[i, "Status horario voo"] = 1    
    elif row['Partida Real'] <= row['Partida Prevista'] + pd.Timedelta(minutes=15): 
        # Voo pontual(0)
        lista_datasets2018.at[i, "Status horario voo"] = 0
    elif pd.isna(row['Partida Real']):
        lista_datasets2018.at[i, "Status horario voo"] = np.nan





In [120]:
lista_datasets2018.head()

,ICAO Empresa Aérea,Número Voo,Código DI,Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Status horario voo
2518,AZU,2018,0.0,N,SBEG,SBBV,2018-12-15 22:00:00,2018-12-15 22:00:00,2018-12-15 23:20:00,2018-12-15 23:20:00,REALIZADO,0.0
2519,AZU,2018,0.0,N,SBKP,SBEG,2018-12-15 17:35:00,2018-12-15 17:31:00,2018-12-15 21:20:00,2018-12-15 21:17:00,REALIZADO,0.0
2520,AZU,2019,7.0,N,SBBR,SBKP,2018-12-16 05:05:00,2018-12-16 05:01:00,2018-12-16 06:40:00,2018-12-16 06:31:00,REALIZADO,0.0
2521,AZU,2019,9.0,N,SBBV,SBBR,2018-12-16 01:05:00,2018-12-16 01:05:00,2018-12-16 04:35:00,2018-12-16 04:35:00,REALIZADO,0.0
2522,AZU,2400,0.0,N,SBGR,SBRJ,2018-12-01 06:05:00,NaT,2018-12-01 07:10:00,NaT,CANCELADO,NaN


In [121]:
lista_datasets2018.isnull().sum()

ICAO Empresa Aérea           0
Número Voo                   0
Código DI                    0
Código Tipo Linha            0
ICAO Aeródromo Origem        0
ICAO Aeródromo Destino       0
Partida Prevista             0
Partida Real              1175
Chegada Prevista             0
Chegada Real              1175
Situação Voo                 0
Status horario voo        1175
dtype: int64

INICIANDO ANALISE EXPLORATORIA

In [122]:
lista_datasets2018.describe()

,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Status horario voo
count,62943,61768,62943,61768,61768.000000
mean,2018-12-16 09:10:01.635765504,2018-12-16 10:34:45.571169792,2018-12-16 10:56:53.600877056,2018-12-16 12:18:35.626214144,0.231787
min,2018-12-01 00:05:00,2018-12-01 00:05:00,2018-12-01 01:10:00,2018-12-01 01:55:00,0.000000
25%,2018-12-08 16:25:00,2018-12-08 17:39:45,2018-12-08 18:12:30,2018-12-08 19:28:15,0.000000
50%,2018-12-16 12:00:00,2018-12-16 13:42:30,2018-12-16 13:45:00,2018-12-16 15:20:30,0.000000
75%,2018-12-23 17:30:00,2018-12-23 19:15:00,2018-12-23 19:10:00,2018-12-23 20:45:30,0.000000
max,2019-01-01 01:25:00,2019-01-01 02:58:00,2019-01-01 03:10:00,2019-01-01 04:03:00,1.000000
std,NaN,NaN,NaN,NaN,0.421977


In [ ]:



# Substitua 'seu_dataframe' pelo nome do seu DataFrame
# df = seu_dataframe

# 1. Contagem de Valores Únicos
for coluna in df.columns:
    print(f"{coluna}: {df[coluna].nunique()} valores únicos")

# 2. Estatísticas Descritivas
desc_stats = df[['Partida Real', 'Chegada Real']].describe()
print(desc_stats)

# 3. Tratamento de Valores Ausentes
df.dropna(subset=['Partida Real', 'Chegada Real'], inplace=True)

# 4. Análise Temporal
df['Partida Prevista'] = pd.to_datetime(df['Partida Prevista'])
df['Chegada Prevista'] = pd.to_datetime(df['Chegada Prevista'])

# 5. Análise de Correlação
correlation_matrix = df[['Partida Real', 'Chegada Real']].corr()
print(correlation_matrix)

cont_historico: historico de voos, contador